In [5]:
import math
import scipy.integrate as integrate


# trapezoid formula: 
$$ I_n = h \cdot \left(\frac{f(a) + f(b)}{2} + \sum_{i=1}^{n-1} f(a + ih) \right), h = \frac{b - a}{n} $$

# Simpson's formula:
$$ I_{Simps, n} = \sum_{i=0}^{\frac{n}{2} - 1} \frac{h}{3} \cdot (f(x_{2i}) + 4f(x_{2i+1}) + f(x_{2i+2})) $$
 Convert it into appropriate view:
$$ \frac{h}{3} (f_0 + f_1 + f_2) = \frac{4h}{3} \left(\frac{f_0 + f_1}{2} + \frac{f_1 + f_2}{2} \right) - \frac{1}{3} \cdot 2h\frac{f_0 + f_2}{2} = 
\frac{4 I_{trap, 2} - I_{trap, 1}}{3}$$

# Runge's rule:
$$ \delta_{2n} = |I_{2n} - I_{n}| $$

# Romberg's method:
Elements Of matrix: $$ I_{i, j} = I_{i, j - 1} - \frac{I_{i, j-1} - I_{i-1, j-1}}{1 - 2^j} $$

In [3]:
class Integration:
    __sum = 0.0
    __nseg = 1
    __ncalls = 0 # считает число вызовов интегрируемой функции

    def __restart(func, x0, x1, nseg0, reset_calls = True):
        """Возвращает интеграл методом трапеций на начальном разбиении"""
        if reset_calls:
            Integration.__ncalls = 0
        Integration.__nseg = nseg0
        # вычисление суммы для метода трапеций с начальным числом отрезков разбиения nseg0
        Integration.__sum = 0.5 * (func(x0) + func(x1))
        dx = 1.0 * (x1 - x0) / nseg0
        for i in range(1, nseg0):
            Integration.__sum += func(x0 + i * dx)

        Integration.__ncalls += 1 + nseg0
        return Integration.__sum * dx

    def __double_nseg(func, x0, x1):
        """Вдвое измельчает разбиение.
           Возвращает интеграл методом трапеций на новом разбиении"""
        nseg = Integration.__nseg
        dx = (x1 - x0) / nseg
        x = x0 + 0.5 * dx
        i = 0
        AddedSum = 0.0
        for i in range(nseg):
            AddedSum += func(x + i * dx)

        Integration.__sum += AddedSum
        Integration.__nseg *= 2
        Integration.__ncalls += nseg
        return Integration.__sum * 0.5 * dx

    def trapezoid(func, x0, x1, epsilon = 1e-10, nseg0 = 1):
        """Интегрирование методом трапеций с заданной точностью и с вычислением погрешности по правилу Рунге.
           nseg0 - число отрезков начального разбиения"""
        ans = Integration.__restart(func, x0, x1, nseg0)
        old_ans = 0.0
        err_est = max(1, abs(ans))
        while (err_est > abs(epsilon * ans)):
            old_ans = ans
            ans = Integration.__double_nseg(func, x0, x1)
            err_est = abs(old_ans - ans)

        print("Total function calls: " + str(Integration.__ncalls))
        return ans

    def simpson(func, x0, x1, epsilon = 1.0e-10, nseg0 = 1):
        """nseg0 - число отрезков начального разбиения"""
        old_trapez_sum = Integration.__restart(func, x0, x1, nseg0)
        new_trapez_sum = Integration.__double_nseg(func, x0, x1)
        ans = (4 * new_trapez_sum - old_trapez_sum) / 3
        old_ans = 0.0
        err_est = max(1, abs(ans))
        while (err_est > abs(epsilon * ans)):
            old_ans = ans
            old_trapez_sum = new_trapez_sum
            new_trapez_sum = Integration.__double_nseg(func, x0, x1)
            ans = (4 * new_trapez_sum - old_trapez_sum) / 3
            err_est = abs(old_ans - ans)

        print("Total function calls: " + str(Integration.__ncalls))
        return ans
    def romberg(func, x0, x1, epsilon = 1e-10, nseg0 = 1, maxcol = 5, reset_calls = True):
        """Интегрирование методом Ромберга
           nseg0 - начальное число отрезков разбиения
           maxcol - максимальный столбец таблицы"""
        # инициализация таблицы
        Itable = [[Integration.__restart(func, x0, x1, nseg0, reset_calls)]]
        i = 0
        maxcol = max(0, maxcol)
        ans = Itable[i][i]
        error_est = max(1, abs(ans))
        while (error_est > abs(epsilon * ans)):
            old_ans = ans
            i += 1
            d = 4.0
            ans_col = min(i, maxcol)
            Itable.append([Integration.__double_nseg(func, x0, x1)] * (ans_col + 1))
            for j in range(0, ans_col):
                diff = Itable[i][j] - Itable[i - 1][j]
                Itable[i][j + 1] = Itable[i][j] + diff / (d - 1.0)
                d *= 4.0

            ans = Itable[i][ans_col]
            if (maxcol <= 1):
                error_est = abs(ans - Itable[i - 1][-1])
            elif (i > maxcol):
                error_est = abs(ans - Itable[i][min(i - maxcol - 1, maxcol - 1)])
            else:
                error_est = abs(ans - Itable[i - 1][i - 1])

        print("Total function calls: " + str(Integration.__ncalls))
        return ans

# Trapezoid integration method applied to:
$ f(x) = \frac{\sin (100x)}{x+1} $

In [6]:
>>> Integration.trapezoid(lambda x: math.sin(100*x) / (x + 1), 0, 1, epsilon=1e-6)

Total function calls: 65537


0.0056992796883593095

# Simpson's integration method to:
$$ f(x) = \frac{\sin (100x)}{x+1} $$

In [8]:
>>> Integration.simpson(lambda x: math.sin(100*x) / (x + 1), 0, 1, epsilon=1e-6)

Total function calls: 2049


0.005699280967919138

# Romberg's integration method (using SciPy library):
$$ f(x) = \frac{\sin (100x)}{x+1} $$

In [7]:
print(integrate.romberg(lambda x: math.sin(100*x) / (x + 1), 0, 1))

0.005699280789554992


# Richardson extrapolation + trapezoid integration:
$$ f(x) = \frac{\sin (100x)}{x+1} $$

In [7]:
>>> Integration.romberg(lambda x: math.sin(100*x) / (x + 1), 0, 1, epsilon=1e-6, maxcol = 2)

Total function calls: 1025


0.0056992807792545916